In [4]:
import os
import numpy as np

directory = "."

ds_names = {"Diabetes","California housing","Liver disorders"}
all_models = []
dat = {} 
for file in os.listdir(directory):
    if file.endswith(".txt"): 
        with open(os.path.join(directory, file),"r") as f:
            for line in f:
                for dname in ds_names:
                    start = line.find(dname)
                    if start > -1:
                        model_name = line[:start].strip()
                        all_models.append(model_name)
                        res = line[start + len(dname):].strip().split()
                        deep = res[0]
                        r2 = res[2]
                        r1 = res[1]
                        dat.setdefault(dname,{}).setdefault(deep,{}).setdefault(model_name,[r1,r2])

all_models = set(all_models)
new_dat = []

for d in dat:
    for dep in dat[d]:
        tmp = [d]
        for model in all_models:
            if model in dat[d][dep]:
                tmp += [np.sqrt(float(dat[d][dep][model][0])), float(dat[d][dep][model][1])]
            else:
                tmp += [None,None]
        new_dat.append(tmp)    


                        

In [5]:
import pandas as pd
from functools import partial

def bold_formatter(x, values, num_decimals=2):
    match = False
    for v in values:
         if round(x, num_decimals) == round(v, num_decimals):
             match = True
             break
             
    if match:
        return f"\\textbf{{{x:.{num_decimals}f}}}"
    else:
        return f"{x:.{num_decimals}f}"

pd.options.display.float_format = '{:.2f}'.format

names_top = [model for model in all_models for _ in (0, 1)]
names_bottom = [i for _ in all_models for i in ("RMSE", "MAE")]

df = pd.DataFrame(new_dat, columns=[[""] + names_top, ["Dataset"] + names_bottom])

for i,c in enumerate(df.columns):
    if i > 1:
        df[c] = pd.to_numeric(df[c])

df = df.set_index([("","Dataset")]).sort_index()

col2_bold = {}

for i,r in df.iterrows():
    ma_odd = r[0::2].min() 
    ma_ev =  r[1::2].min()
    ma_odd_id = 2 * r[0::2].argmin()
    ma_ev_id = 2 * r[1::2].argmin() + 1
    ma_odd = r[ma_odd_id]
    ma_ev = r[ma_ev_id]
    if not df.columns[ma_odd_id] in col2_bold:
        col2_bold[df.columns[ma_odd_id]] = []
    col2_bold[df.columns[ma_odd_id]].append(ma_odd)
    if not df.columns[ma_ev_id] in col2_bold:
        col2_bold[df.columns[ma_ev_id]] = []
    col2_bold[df.columns[ma_ev_id]].append(ma_ev)
    
fmts_max_2f = {column: partial(bold_formatter, values=col2_bold[column], num_decimals=2) for column in col2_bold}



/tmp/ipykernel_2105430/3617188221.py:36: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ma_odd = r[ma_odd_id]
/tmp/ipykernel_2105430/3617188221.py:37: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ma_ev = r[ma_ev_id]


In [6]:
print(df.to_latex(float_format="%.2f",formatters=fmts_max_2f))

\begin{tabular}{lrrrrrrrrrr}
\toprule
 & \multicolumn{2}{r}{Cascade Forest} & \multicolumn{2}{r}{XGB} & \multicolumn{2}{r}{Boosted Forest} & \multicolumn{2}{r}{ecdfr} & \multicolumn{2}{r}{AWDF} \\
 & RMSE & MAE & RMSE & MAE & RMSE & MAE & RMSE & MAE & RMSE & MAE \\
('', 'Dataset') &  &  &  &  &  &  &  &  &  &  \\
\midrule
California housing & 0.75 & 0.57 & 0.66 & 0.48 & \textbf{0.49} & \textbf{0.33} & 0.84 & 0.64 & 0.76 & 0.56 \\
Diabetes & 52.35 & \textbf{42.01} & 53.08 & 42.63 & \textbf{52.34} & 42.39 & 52.89 & 42.84 & 52.99 & 42.21 \\
Liver disorders & 2.87 & 2.32 & 2.96 & 2.40 & 2.86 & \textbf{2.31} & \textbf{2.84} & 2.36 & 3.58 & 2.54 \\
\bottomrule
\end{tabular}



In [7]:
df

Cascade Forest         XGB       Boosted Forest        \
                             RMSE   MAE  RMSE   MAE           RMSE   MAE   
(, Dataset)                                                                
California housing           0.75  0.57  0.66  0.48           0.49  0.33   
Diabetes                    52.35 42.01 53.08 42.63          52.34 42.39   
Liver disorders              2.87  2.32  2.96  2.40           2.86  2.31   

                   ecdfr        AWDF        
                    RMSE   MAE  RMSE   MAE  
(, Dataset)                                 
California housing  0.84  0.64  0.76  0.56  
Diabetes           52.89 42.84 52.99 42.21  
Liver disorders     2.84  2.36  3.58  2.54